In [8]:
import mmap
import os
import time
import numpy as np

regset_gpio = np.dtype([
    ('o', 'uint32'),
    ('t', 'uint32'),
    ('i', 'uint32')
])

os.system('cat red_pitaya.bit > /dev/xdevcfg')

fd = os.open('/dev/uio/api', os.O_RDWR)
m = mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x0)
gpio_array = np.recarray(1, regset_gpio, buf=m)
gpio = gpio_array[0]

# enable output
gpio.t = 0x000000

# blink
for i in range(10):
    gpio.o = 0xf00000
    time.sleep(0.2)
    gpio.o = 0x0f0000
    time.sleep(0.2)

In [9]:
hex(gpio.o)

'0xf0000'

In [10]:
hex(gpio.i)

'0xf0000'